# Analyzing Trends in Video Games on Steam (((((((WIP))))))))
by Sean Ha

# Introduction (((((WIP))))))))
The video games industry is larger than it has ever been, with a global market value of over \\$100 billion. In 2021, it is expected to reach approximately \\$140 billion. There are various devices through which these games are distributed, including computers, game consoles, and smartphones. For each of these devices, there usually exist various storefronts through which games are sold. Examples of this include the Apple App Store for Apple devices, the Google Play Store for Android devices, the PlayStation Store for PlayStation consoles, and Steam for PCs. While there exist many other storefronts for PCs, we will be focusing our analysis on Steam, as it is currently the largest PC digital distributor for games.

Developing video games takes lots of time and money. If we could analyze data of existing games, we could perhaps reach conclusions about what types of games would be the most successful, or profitable, and minimize the chances of developing a game that will not sell particularly well. If you were a game designer or a manager at a game studio, you very well might be interested in this sort of data.

In recent times, through technological advancements and innovations, the barrier of entry for making a game has been drastically lowered. As a result, there has been a surge of "indie" game developers (usually classified by a small development team, that does not have the financial support of a large game publisher). These indie game developers are usually short on cash, and are relying on the success of their project to stay afloat. This type of data might be very useful to these kinds of people, as it could help them make an informed decision about what type of game to develop, and specific factors to think about, as opposed to taking a shot in the dark.

# Setup ((WIP))))))))))))))))
Below are the libraries that we will be needing for this project. These will help us store and analyze our data.

In [92]:
import pandas as pd
import numpy as np

# Loading the Data
We are using a CSV file that contains the dataset. The dataset and a description of what it contains can be found here: https://data.world/craigkelly/steam-game-data. Please take a look at that page to understand what particular columns in the dataset might represent, and where the data came from.

We will load the dataset into a Pandas DataFrame. A DataFrame is a 2 dimensional data structure, similar to something like an SQL table. It will help us in organizing and analyzing our data.

In [33]:
df = pd.read_csv("games-features.csv")
df.head()

,QueryID,ResponseID,QueryName,ResponseName,ReleaseDate,RequiredAge,DemoCount,DeveloperCount,DLCCount,Metacritic,...,LegalNotice,Reviews,SupportedLanguages,Website,PCMinReqsText,PCRecReqsText,LinuxMinReqsText,LinuxRecReqsText,MacMinReqsText,MacRecReqsText
0,10,10,Counter-Strike,Counter-Strike,Nov 1 2000,0,0,1,0,88,...,,,English French German Italian Spanish Simplifi...,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
1,20,20,Team Fortress Classic,Team Fortress Classic,Apr 1 1999,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
2,30,30,Day of Defeat,Day of Defeat,May 1 2003,0,0,1,0,79,...,,,English French German Italian Spanish,http://www.dayofdefeat.com/,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
3,40,40,Deathmatch Classic,Deathmatch Classic,Jun 1 2001,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
4,50,50,Half-Life: Opposing Force,Half-Life: Opposing Force,Nov 1 1999,0,0,1,0,0,...,,,English French German Korean,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,


# Tidy Data



We will drop the unnecessary columns that we will not need in this project. For us, that includes the following:

- PriceCurrency
- SupportEmail
- SupportURL
- AboutText
- Background
- ShortDescrip
- DetailedDescrip
- DRMNotice
- ExtUserAcctNotice
- HeaderImage
- LegalNotice
- Reviews
- Website
- PCMinReqsText
- PCRecReqsText
- LinuxMinReqsText
- LinuxRecReqsText
- MacMinReqsText
- MacRecReqsText

We will remove these columns in particular because they generally contain information that is either meaningless to analyze, or simply not relevant to this project.

In [34]:
df = df.drop(['PriceCurrency', 'SupportEmail', 'SupportURL', 'AboutText', 'Background', 'ShortDescrip', 
              'DetailedDescrip', 'DRMNotice','ExtUserAcctNotice', 'HeaderImage', 'LegalNotice', 'Reviews', 
              'Website', 'PCMinReqsText', 'PCRecReqsText', 'LinuxMinReqsText', 'LinuxRecReqsText', 
              'MacMinReqsText', 'MacRecReqsText'], axis=1)
df.head()

Now let's clean up the "SupportedLanguages" column. Currently, some observations contain unnecessary information in that column. What we what to extract is the number of languages that a particular game supports. In addition, we will want to take a look at some major languages, and see what impact they might have on the success of a game. We could hypothetically perform this on every single language, but since there are so many, we will limit our analysis to a few major ones. If you are interested in a particular language that I do not include, feel free to try it out yourself and see what kind of results you get!

What we will do is split this column up into some new ones. One for the number of languages the game supports ("LangSupportedCount"). And then additional columns for whether or not a particular game supports a particular language (e.g. a column named "SupportsEnglish" would contain 1 if the game support English, and 0 if it does not).

This will separate different data from one column into multiple columns, making it easier for analysis in the future. We want multiple columns to contain specific information, as opposed to having one column with a large amount of information.

For this project, here are the languages that we will create specific columns for:
- English
- French
- Spanish
- Japanese
- Chinese (1 if game supports traditional, simplified, or both; 0 if neither)
- Korean

In [81]:
# Create column for number of languages support
df['LangSupportCount'] = 0

# Create columns for the specific languages specified above
df['SupportsEnglish'] = 0
df['SupportsFrench'] = 0
df['SupportsSpanish'] = 0
df['SupportsJapanese'] = 0
df['SupportsChinese'] = 0
df['SupportsKorean'] = 0

for index, row in df.iterrows():
    # Remove unnecessary data (asterisks, the string "languages with full audio support")
    langs = df.iloc[index].SupportedLanguages
    df.loc[index, 'SupportedLanguages'] = langs.replace('*', '')
    langs = df.iloc[index].SupportedLanguages
    df.loc[index, 'SupportedLanguages'] = langs.replace('languages with full audio support', '')
    
    # Put value for number of languages supported
    lang_list = df.iloc[index].SupportedLanguages.split()
    df.loc[index, 'LangSupportCount'] = len(lang_list)
    
    # Converts languages to all uppercase for case insensitivity
    lang_list = [lang.upper() for lang in lang_list]
    
    # Put values for whether or not game supports each language
    if 'ENGLISH' in lang_list:
        df.loc[index, 'SupportsEnglish'] = 1
    if 'FRENCH' in lang_list:
        df.loc[index, 'SupportsFrench'] = 1
    if 'SPANISH' in lang_list:
        df.loc[index, 'SupportsSpanish'] = 1
    if 'JAPANESE' in lang_list:
        df.loc[index, 'SupportsJapanese'] = 1
    if 'TRADITIONAL CHINESE' in lang_list or 'SIMPLIFIED CHINESE' in lang_list:
        df.loc[index, 'SupportsChinese'] = 1
    if 'KOREAN' in lang_list:
        df.loc[index, 'SupportsKorean'] = 1
        
# Drop the now unnecessary "SupportedLanguages" column
df = df.drop(['SupportedLanguages'], axis=1)

Next, we will convert the boolean columns into integer columns. So in other words, True becomes 1, and False becomes 0.  This will allow us to perform operations on these columns that can only be done on numerical values.

In [87]:
for col in df.columns:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)

# Missing Data
In many instances, your data might contain missing values. You will have to decide how you want to handle such missing data. We will check this dataset to see if there are any missing values.

In [91]:
# Find number of NaN values in all columns
missing_count_df = pd.DataFrame(df.isna().sum(),columns=["Missing Values"])
missing_count_df[(missing_count_df.T != 0).any()]

,Missing Values
QueryName,1


Luckily in our case, we have essentially no missing data. We have one missing value for one observation in our data. One observation has a missing QueryName. Why it is missing is unkown. However, in this dataset, we know that ResponseName is the same as QueryName (or at least very similar). As a result, while this observation is missing a QueryName, it still has a ResponseName, which helps us identify the name of the game. Therefore, we can simply fill in the QueryName for this observation with the ResponseName for the same observation.

In [94]:
# Search for an replace the NaN value with the ResponseName value in the same row
for index, row in df.iterrows():
    if type(row.QueryName) != str and np.isnan(row.QueryName):
        df.loc[index, 'QueryName'] = row.ResponseName
        break

The Metacritic column contains a value which is the "score" that Metacritic gives the game, on a scale from 0 to 100, 100 being the best. An important note to make is that a value of 0 in the Metacritic column should be considered a missing value, as it means that Metacritic does not have a score for the game. Theoretically, it could also mean that the game was given a 0 score, but Metacritic has never given out a score of 0 to a game (to the best of my knowledge). As a result, we will convert all zeroes in the Metacritic column to NaN values, so that these missing values do not skew our data.

In [99]:
# All zeroes in Metacritic column get turned into NaN
for index, row in df.iterrows():
    if row.Metacritic == 0:
        df.loc[index, 'Metacritic'] = np.nan

With this, we have finished tidying our data. We can move on to the next step. Before that, however, I will give a brief description of every column in our DataFrame, and what it means. Hopefully, if you are not familiar with Steam or video games, this will help you get an overview of what the data is actually representing.

- ColName : ColDesc
-

In [100]:
df

,QueryID,ResponseID,QueryName,ResponseName,ReleaseDate,RequiredAge,DemoCount,DeveloperCount,DLCCount,Metacritic,...,GenreIsMassivelyMultiplayer,PriceInitial,PriceFinal,LangSupportCount,SupportsEnglish,SupportsFrench,SupportsSpanish,SupportsJapanese,SupportsChinese,SupportsKorean
0,10,10,Counter-Strike,Counter-Strike,Nov 1 2000,0,0,1,0,88.0,...,0,9.99,9.99,10,1,1,1,0,0,1
1,20,20,Team Fortress Classic,Team Fortress Classic,Apr 1 1999,0,0,1,0,NaN,...,0,4.99,4.99,5,1,1,1,0,0,0
2,30,30,Day of Defeat,Day of Defeat,May 1 2003,0,0,1,0,79.0,...,0,4.99,4.99,5,1,1,1,0,0,0
3,40,40,Deathmatch Classic,Deathmatch Classic,Jun 1 2001,0,0,1,0,NaN,...,0,4.99,4.99,5,1,1,1,0,0,0
4,50,50,Half-Life: Opposing Force,Half-Life: Opposing Force,Nov 1 1999,0,0,1,0,NaN,...,0,4.99,4.99,4,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13352,567660,567660,Baseball Riot,Baseball Riot,Jan 17 2017,0,0,1,0,NaN,...,0,0.00,0.00,1,1,0,0,0,0,0
13353,567860,567860,Passage 4,Passage 4,Dec 13 2016,0,0,1,0,NaN,...,0,0.00,0.00,6,1,1,1,0,0,0
13354,567940,567940,Piximalism,Piximalism,Sep 26 2019,0,0,1,0,NaN,...,0,0.00,0.00,1,1,0,0,0,0,0
13355,568150,568150,Technoball,Technoball,Dec 12 2016,0,0,1,0,NaN,...,0,0.00,0.00,1,1,0,0,0,0,0


In [86]:
type(df.GenreIsMassivelyMultiplayer)

pandas.core.series.Series

In [84]:
df.columns.unique()

Index(['QueryID', 'ResponseID', 'QueryName', 'ResponseName', 'ReleaseDate',
       'RequiredAge', 'DemoCount', 'DeveloperCount', 'DLCCount', 'Metacritic',
       'MovieCount', 'PackageCount', 'RecommendationCount', 'PublisherCount',
       'ScreenshotCount', 'SteamSpyOwners', 'SteamSpyOwnersVariance',
       'SteamSpyPlayersEstimate', 'SteamSpyPlayersVariance',
       'AchievementCount', 'AchievementHighlightedCount', 'ControllerSupport',
       'IsFree', 'FreeVerAvail', 'PurchaseAvail', 'SubscriptionAvail',
       'PlatformWindows', 'PlatformLinux', 'PlatformMac', 'PCReqsHaveMin',
       'PCReqsHaveRec', 'LinuxReqsHaveMin', 'LinuxReqsHaveRec',
       'MacReqsHaveMin', 'MacReqsHaveRec', 'CategorySinglePlayer',
       'CategoryMultiplayer', 'CategoryCoop', 'CategoryMMO',
       'CategoryInAppPurchase', 'CategoryIncludeSrcSDK',
       'CategoryIncludeLevelEditor', 'CategoryVRSupport', 'GenreIsNonGame',
       'GenreIsIndie', 'GenreIsAction', 'GenreIsAdventure', 'GenreIsCasual',
       'G

In [98]:
0 in df.SteamSpyPlayersEstimate.unique()

True